## Extraktionsbaserad textsammanfattare med olika rankningsmått 

In [70]:
# Imports
import pandas as pd
import numpy as np
import nltk 
import ssl

# Fixes some errors, found online at https://github.com/gunthercox/ChatterBot/issues/930#issuecomment-322111087
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

In [71]:
# Web scrapping -->  module för att ladda ner artiklar 
from newspaper import Article
text = "https://www.aftonbladet.se/nyheter/a/kE6ExL/sd-far-tunga-poster-i-utskotten"
article = Article(text, language='sv')
article.download()
article.parse()
#article.nlp()
#article.summary

text = article.text

# Preprocessing 

## Overview
### List 1 - sentences
* Varje mening separat

### Dataframe - scores
#### columns are the score of each ranking measure
* Baseline
* Headings
* TF/IDF-score
* NER
* ~~Class~~ //Om vi har tid för ML

### List 2 -> Cleaned for Stop Words 
* Varje mening separat 

###

##### List 1

In [72]:
# removes endlines:
from token import NEWLINE


org_sentences = text.replace('\n\n', '. ')
# creates some exceptions from above rule
org_sentences = org_sentences.replace('.. ', '. ')
org_sentences = org_sentences.replace(':. ', ': ')
org_sentences = org_sentences.split('. ')
org_sentences[0:10]

['SD får tunga poster i utskotten',
 'Publicerad: I dag 12.09 Uppdaterad: Mindre än 2 tim sedan',
 'Sverigedemokraterna får ordförandeposten i riksdagens justitie- och utrikesutskott',
 'Nu går ledarna för vänsterblocket till hård attack',
 '– Det är skrämmande, ganska chockartat, säger Socialdemokraternas gruppledare Lena Hallengren till Aftonbladet',
 'Sverigedemokraterna , Moderaterna, Kristdemokraterna och Liberalerna har delat upp posterna i utskotten och EU-nämnden',
 'Där tar Sverigedemokraterna flera viktiga poster',
 'Bland annat tilldelas partiet ordförandeposten i arbetsmarknadsutskottet, näringsutskottet, justitieutskottet samt utrikesutskottet, enligt ett pressmeddelande',
 'De erhåller även posten som vice ordförande i civilutskottet, trafikutskottet, försvarsutskottet samt skatteutskottet',
 '– Det som överraskade mig mest, men som jag kan se varför de vill ha, är ordförandeposten i utrikesutskottet']

##### Dataframe

In [73]:
index = range(org_sentences.__len__())
columns = ['Baseline', 'Headings', 'TF/IDF', 'NER']
scores = pd.DataFrame(index=index, columns=columns)
scores.fillna(0, inplace=True)
scores.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34 entries, 0 to 33
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   Baseline  34 non-null     int64
 1   Headings  34 non-null     int64
 2   TF/IDF    34 non-null     int64
 3   NER       34 non-null     int64
dtypes: int64(4)
memory usage: 1.2 KB


### Stop Word Filtering
* Hur stor korpus ska vi ha? 

In [74]:
from nltk.corpus import stopwords
#nltk.download('stopwords')

swe_stop_words = stopwords.words('swedish')
print(len(swe_stop_words))

# Snowball Swedish Stop Words
snowball_sw = pd.read_csv("resources/stop_words_swedish_snowball .txt")
# print(snowball_sw)

# Borrowed from https://github.com/peterdalle/svensktext/tree/master/stoppord
def get_stopwords(wordlist = "standard"):
    if wordlist == "standard":
        url = "https://raw.githubusercontent.com/peterdalle/svensktext/master/stoppord/stoppord.csv"
    elif wordlist == "many":
        url = "https://raw.githubusercontent.com/peterdalle/svensktext/master/stoppord/stoppord-mycket.csv"
    elif wordlist == "politics":
        url = "https://raw.githubusercontent.com/peterdalle/svensktext/master/stoppord/stoppord-politik.csv"
    else:
        raise ValueError("Argument 'wordlist' must be 'standard', 'many' or 'politics', not '{}'.".format(wordlist))
    return pd.read_csv(url, header=1, encoding="utf-8")

stopwords = get_stopwords()
# print(stopwords)

114


In [75]:
# Perform Stop Word Filtering On Example (Original Text) 
def stop_word_filtering(original_text): 
    swf_text = None
    return swf_text

### Lemmatizer & NER 

* LEmmatizer verkar ok 
* NER --> helt klart bristande -- men det kan duga? 

# REMOVE DUPLICATE ENTITIES 

In [76]:
# Spacy NLP Pipeline
# spaCy + Lemmy (https://github.com/sorenlind/lemmy) ??? --> Testa om det blir bättre täckning (recall)? 
import spacy
#from spacy.lang.sv.examples import sentences 

# Credit to Explosion for sv_core_news_sm --> https://github.com/explosion 

nlp = spacy.load("sv_core_news_sm")

doc = nlp(text)
#print(doc.text)
print(doc.ents) # --> KANSKE ATT DET STORA PAKETET ÄR BÄTTRE
for token in doc:
    print(token)
    #print(token.text, token.pos_, token.dep_)
    print(token.lemma_)
    pass

(Socialdemokraternas, Lena, Aftonbladet, Moderaterna, KD, Aftonbladets My Rohwedder, Aftonbladet, Sveriges, Moderaterna, Ulf Kristerssons, Lena Hallengren, Sverige, Skämmer, Sverige, Märta, Twitter:

, Putin, Biden, Sverige, Twitter.

, Ali, Esbati, Martin Kinnunen, Markus Wiechels, Syrien, Marcus Wiechel, Martin Kinnunen, UD:s, Syrien, Annie, KD, Sverige, Nato, Sveriges, Jakob Hallgren)
SD
SD
får
få
tunga
tung
poster
post
i
i
utskotten
utskott






Publicerad
publicerad
:
:
I
i
dag
dag
12.09
12.09
Uppdaterad
uppdaterad
:
:
Mindre
Mindre
än
än
2
2
tim
tim
sedan
sedan






Sverigedemokraterna
Sverigedemokraterna
får
få
ordförandeposten
ordförandepost
i
i
riksdagens
riksdag
justitie-
justitie
och
och
utrikesutskott
utrikesutskott
.
.






Nu
nu
går
gå
ledarna
ledarna
för
för
vänsterblocket
vänsterblock
till
till
hård
hård
attack
attack
.
.






–
–
Det
den
är
vara
skrämmande
skrämmande
,
,
ganska
ganska
chockartat
chockarta
,
,
säger
säga
Socialdemokraternas
Socialdemokraterna
gruppl

In [77]:
# Precision & Recall 
def calc_p_r(text_output): 
    pass

In [78]:
# Lemmatizer 
# Stanza https://stanfordnlp.github.io/stanza/installation_usage.html + Språkbanken https://spraakbanken.gu.se/en/resources/stanzalem
# https://nlp.johnsnowlabs.com/2020/05/05/lemma_sv.html

# Ranking Measures

In [79]:
# Index org text sentences 

### Baseline
(1/N --> n=ordning mening kommer i dvs. första meningen får N=1 -> 1/1, andra meningen får N=2 -> 1/2, osv.)

In [89]:
# Ranking metric 1 --> Baseline
for i, score in enumerate(scores['Baseline']) :
    scores['Baseline'][i] = 1/((i+1)**0.5)
scores.describe()

,Baseline,Headings,TF/IDF,NER
count,34.000000,34.0,34.0,34.0
mean,0.302561,0.0,0.0,0.0
std,0.174576,0.0,0.0,0.0
min,0.171499,0.0,0.0,0.0
25%,0.197087,0.0,0.0,0.0
50%,0.239119,0.0,0.0,0.0
75%,0.329057,0.0,0.0,0.0
max,1.000000,0.0,0.0,0.0


### Headings

In [ ]:
# Ranking metric 2 --> Headings 

### TF*IDF-score
* Diskutera hur vi kan använda måtten 
* If similarity is close --> Similar content --> Remove redundance?  



https://forketyfork.medium.com/latex-math-formulas-a-cheat-sheet-21e5eca70aae

In [ ]:
# Ranking metric 3 --> TF*IDF

# Term Weights --> Calculate importance of single words in text/doc
# Binary term weights --> document specific
# TF*IDF term weights --> document-collection specific 

# Assign weights to each dimension (attr/word) of each sentence (record/example) 

# Term Frequency (TF-score) --> TFij == frequency of the jth term in in the ith doc 

# Inverse Document Frequency 
# idf-score of the jth term measures the uniqueness of the jth term in the collection of documents
# IDFj = log(M / Nj)
#
# M = total num of docs in collection 
# Nj is the number of documents that contain the jth term

# HIGH TF*IDF-score 
# Word frequent in document && Occur in few documents of the collection 
# LOW TF*IDF-score
# Not present in document || present in all documents of the collection 

### NER 
* (nltk lib) --> (Meningar med Named Entities är troligtvis viktigare)

In [ ]:
# https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/streamlit_notebooks/NER.ipynb#scrollTo=9RgiqfX5XDqb
# SPARK NLP 
import sparknlp
spark = sparknlp.start(m1=True)

# Combination Function

## Overview
### Standardize
* standardize all scores
### Combine
* Combine the scores into one overall score
* add weight and/or ML if time allows

In [ ]:
# Combination Function
# Här ligger ML om vi gör det  

# Assemble output 
* Reassemble according to overall score ranking
* Output summarization 

In [ ]:
# Assemble Output 

In [ ]:
# Run 